In [1]:
import glob
import os
import numpy as np
from PIL import Image
import skimage
import skimage.io
import scipy.misc
import imageio
import warnings; warnings.simplefilter('ignore')

list_classes = [
 'Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']

list_dict = {}
for i in range(10):
    key = list_classes[i]
    v = [0,0,0,0,0,0,0,0,0,0]
    v[i] = 1
    list_dict[key] = v
print(list_dict)

train_dir = '../input/crop_train'
test_dir = '../input/test'
test_files = sorted(glob.glob(test_dir+'/*'))
train_files = sorted(glob.glob(train_dir+'/*/*'))
train_data_cnt = len(train_files)
from sklearn.utils import shuffle
train_files = shuffle(train_files,random_state=233)
train_cnt = int(train_data_cnt * 0.8)
valid_cnt = train_data_cnt - train_cnt
BATCH_SIZE = 32
CROP_LEN = 112

def random_crop(im_array):
    # crop
    x_range = im_array.shape[0] - CROP_LEN
    y_range = im_array.shape[1] - CROP_LEN
    # print(x_range,y_range)
    a = np.random.randint(x_range)
    b = a + CROP_LEN
    c = np.random.randint(y_range)
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array

def center_crop(im_array):
    center_x = im_array.shape[0] // 2
    center_y = im_array.shape[1] // 2
    half_crop = CROP_LEN // 2
    a = center_x - half_crop
    b = a + CROP_LEN
    c = center_y - half_crop
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array
    

def get_img(img_path, train_flag = True):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    
    # train or valid
    if train_flag is True:
        final_img = random_crop(im_array)
    else:
        final_img = center_crop(im_array)
    
    final_img = final_img/255.0
    return final_img

# 读取测试数据中间 224
def get_test_img(img_path):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    final_img = center_crop(im_array)
    final_img = final_img/255.0
    return final_img

def get_y(img_path):
    n = img_path.split('/')[-2]
    return list_dict[n]

def data_gen(file_list, batch_size=BATCH_SIZE, train_flag = True):
    curr_idx = 0
    data_cnt = len(file_list)
    while True:
        if curr_idx + batch_size > data_cnt:
            start_idx = data_cnt-batch_size
            end_idx = data_cnt
            curr_idx = 0
        else:
            start_idx = curr_idx
            end_idx = curr_idx + batch_size
            curr_idx += batch_size
        curr_fl = file_list[start_idx:end_idx]
        curr_x = [get_img(p,train_flag) for p in curr_fl]
        curr_x = np.array(curr_x,dtype='float32')
        curr_y = np.array([get_y(p) for p in curr_fl])
        yield curr_x,curr_y

train_gen = data_gen(train_files[:train_cnt], BATCH_SIZE, True)
valid_gen = data_gen(train_files[train_cnt:], BATCH_SIZE, False)
import math
train_step = math.ceil(train_cnt*0.5/BATCH_SIZE)
valid_step = math.ceil(valid_cnt/BATCH_SIZE)
    

{'Motorola-Nexus-6': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 'LG-Nexus-5x': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'iPhone-6': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 'Motorola-X': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'Sony-NEX-7': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'HTC-1-M7': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 'Samsung-Galaxy-S4': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 'Samsung-Galaxy-Note3': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 'Motorola-Droid-Maxx': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 'iPhone-4s': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]}


In [2]:
# def model
from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


In [3]:
model_p = 'best_xception_crop.h5'  # copy less manip xception

if os.path.exists(model_p):
    model = load_model(model_p)
    K.set_value(model.optimizer.lr, 0.0001)
    print('load',model_p)

load best_xception_crop.h5


In [4]:
model_chk = ModelCheckpoint(filepath=model_p, monitor='val_acc', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, verbose=1,
                              patience=3, min_lr=0.00000001)

model.fit_generator(train_gen,
          steps_per_epoch = train_step,
          epochs=40,
          validation_data = valid_gen,
          validation_steps = valid_step,
          callbacks=[model_chk,reduce_lr]
         )
           

Epoch 1/40
550/550 [==============================] - 333s 606ms/step - loss: 0.5606 - acc: 0.8101 - val_loss: 0.6073 - val_acc: 0.7911
Epoch 2/40
550/550 [==============================] - 323s 587ms/step - loss: 0.5622 - acc: 0.8106 - val_loss: 0.6356 - val_acc: 0.7818
Epoch 3/40
550/550 [==============================] - 324s 590ms/step - loss: 0.5428 - acc: 0.8139 - val_loss: 0.5398 - val_acc: 0.8194
Epoch 4/40
550/550 [==============================] - 324s 589ms/step - loss: 0.5400 - acc: 0.8197 - val_loss: 0.7193 - val_acc: 0.7575
Epoch 5/40
550/550 [==============================] - 325s 590ms/step - loss: 0.5208 - acc: 0.8231 - val_loss: 0.6103 - val_acc: 0.7972
Epoch 6/40
550/550 [==============================] - 324s 590ms/step - loss: 0.5181 - acc: 0.8239 - val_loss: 0.5559 - val_acc: 0.8152
Epoch 7/40
550/550 [==============================] - 325s 591ms/step - loss: 0.5151 - acc: 0.8274 - val_loss: 0.5168 - val_acc: 0.8228
Epoch 8/40
550/550 [============================

Epoch 32/40
550/550 [==============================] - 322s 585ms/step - loss: 0.3678 - acc: 0.8769 - val_loss: 0.5325 - val_acc: 0.8311
Epoch 33/40
550/550 [==============================] - 326s 592ms/step - loss: 0.3635 - acc: 0.8757 - val_loss: 0.4241 - val_acc: 0.8631
Epoch 34/40
550/550 [==============================] - 322s 586ms/step - loss: 0.3593 - acc: 0.8801 - val_loss: 0.4337 - val_acc: 0.8577
Epoch 35/40
550/550 [==============================] - 323s 587ms/step - loss: 0.3486 - acc: 0.8836 - val_loss: 0.5399 - val_acc: 0.8210
Epoch 36/40
550/550 [==============================] - 323s 588ms/step - loss: 0.3469 - acc: 0.8807 - val_loss: 0.4487 - val_acc: 0.8492
Epoch 37/40
550/550 [==============================] - 323s 587ms/step - loss: 0.3474 - acc: 0.8823 - val_loss: 0.4113 - val_acc: 0.8659
Epoch 38/40
550/550 [==============================] - 324s 589ms/step - loss: 0.3411 - acc: 0.8845 - val_loss: 0.3979 - val_acc: 0.8661
Epoch 39/40
550/550 [====================